In [4]:
import pandas as pd
import numpy as np

In [5]:
# Задание 1 про возможность использовать метод na_values тоже уяснила

fail = pd.read_csv('horse_data.csv', names = ['surgery', 'age', 'number','4', 'puls','6', '7', '8', '9', '10', '11', 'perestalsis', 'distension', '14', '15', '16', '17', '18', '19', '20', '21', '22', 'outcome', '24', '25', '26', '27', '28',], sep = ',', header=None)
fail.head()

,surgery,age,number,4,puls,6,7,8,9,10,...,19,20,21,22,outcome,24,25,26,27,28
0,2,1,530101,38.50,66,28,3,3,?,2,...,45.00,8.40,?,?,2,2,11300,0,0,2
1,1,1,534817,39.2,88,20,?,?,4,1,...,50,85,2,2,3,2,2208,0,0,2
2,2,1,530334,38.30,40,24,1,1,3,1,...,33.00,6.70,?,?,1,2,0,0,0,1
3,1,9,5290409,39.10,164,84,4,1,6,2,...,48.00,7.20,3,5.30,2,1,2208,0,0,1
4,2,1,530255,37.30,104,35,?,?,6,2,...,74.00,7.40,?,?,2,2,4300,0,0,2


In [6]:
data = fail.loc[:, ['surgery', 'age', 'number','puls', 'perestalsis', 'distension', 'outcome']]
new_df = data.replace('?', np.nan)
df = new_df.apply(lambda column: pd.to_numeric(column), axis=0)
df.head()

,surgery,age,number,puls,perestalsis,distension,outcome
0,2.0,1,530101,66.0,4.0,4.0,2.0
1,1.0,1,534817,88.0,4.0,2.0,3.0
2,2.0,1,530334,40.0,3.0,1.0,1.0
3,1.0,9,5290409,164.0,4.0,4.0,2.0
4,2.0,1,530255,104.0,NaN,NaN,2.0


In [35]:
df.shape

(300, 7)

In [36]:
df.describe()

,surgery,age,number,puls,perestalsis,distension,outcome
count,299.000000,300.000000,3.000000e+02,300.000000,256.000000,244.000000,299.000000
mean,1.397993,1.640000,1.085889e+06,71.280000,2.917969,2.266393,1.551839
std,0.490305,2.173972,1.529801e+06,27.541545,0.976744,1.065131,0.737187
min,1.000000,1.000000,5.184760e+05,30.000000,1.000000,1.000000,1.000000
25%,1.000000,1.000000,5.289040e+05,48.000000,3.000000,1.000000,1.000000
50%,1.000000,1.000000,5.303055e+05,64.000000,3.000000,2.000000,1.000000
75%,2.000000,1.000000,5.347275e+05,88.000000,4.000000,3.000000,2.000000
max,2.000000,9.000000,5.305629e+06,184.000000,4.000000,4.000000,3.000000


In [37]:
# Задание 2
# Выбросы через межквартльный размах (на примере данных о пульсе)

q1 = df.puls.quantile(0.25)
q3 = df.puls.quantile(0.75)
iqr = q3 - q1
lower_bound = q1 - (1.5 * iqr) 
upper_bound = q3 + (1.5 * iqr)
remove_outliers = df[df.puls.between(lower_bound, upper_bound, inclusive=True)].sort_values('puls')
remove_outliers

,surgery,age,number,puls,perestalsis,distension,outcome
232,1.0,1,5262543,30.0,NaN,NaN,1.0
130,1.0,1,528964,30.0,NaN,NaN,2.0
203,2.0,1,529685,36.0,3.0,1.0,1.0
276,1.0,1,534787,36.0,2.0,2.0,3.0
242,1.0,1,530354,36.0,3.0,3.0,2.0
...,...,...,...,...,...,...,...
148,1.0,1,529272,132.0,4.0,2.0,1.0
135,1.0,9,5287179,136.0,1.0,3.0,2.0
44,1.0,1,535407,140.0,4.0,NaN,3.0
103,1.0,9,5292489,140.0,3.0,2.0,1.0


In [27]:
print('Значение моды для пульса: ', df.puls.round().mode()[0])
print('Среднеарифметическое для пульса: ', df.puls.mean())

Значение моды для пульса:  48.0
Среднеарифметическое для пульса:  71.91304347826087


In [29]:
# remove_outliers.describe()

In [69]:
# Что это за выброс?
"""
Среднее значение пульса 72 удара, мода составила 48 ударов. К выбросам были отнесены значения 150 и более ударов в минуту
(более, чем в 2 раза больше среднего и более, чем в три раза больше моды). При этом из описательной части датасета мы знаем,
что нормальным для взрослых лошадей считается пульс от 30-40 ударов в минуту, для животных с патологией пульс может
быть больше (перед нами база данных именно о таких лошадях). 
Могу предположить, что причиной выбросов стал человеческий фактор (ошибка ввода данных). 
Это очевидные промахи и они малочисленны (доля не превышает 2%).
Не стала бы принимать их в ходе анализа данных к учету.
"""
pd.concat([df, remove_outliers]).drop_duplicates(keep=False)

,surgery,age,number,puls,perestalsis,distension,outcome
3,1.0,9,5290409,164.0,4.0,4.0,2.0
5,2.0,1,528355,NaN,3.0,2.0,1.0
28,1.0,1,5279442,NaN,NaN,NaN,2.0
41,2.0,9,5288249,150.0,NaN,NaN,1.0
52,2.0,1,529483,NaN,3.0,1.0,1.0
55,1.0,9,5282839,160.0,3.0,4.0,2.0
56,1.0,1,528872,NaN,NaN,NaN,1.0
58,1.0,1,528298,NaN,4.0,3.0,2.0
74,1.0,9,5292929,NaN,NaN,NaN,2.0
93,2.0,1,530310,NaN,3.0,3.0,2.0


In [81]:
# Задание 3
#Доля пропусков для всех выбранных столбцов

for col in df.columns:
    pct_missing = df[col].isnull().mean()
    print(f'{col} - {pct_missing :.1%}')   
    

surgery - 0.3%
age - 0.0%
number - 0.0%
puls - 8.0%
perestalsis - 14.7%
distension - 18.7%
outcome - 0.3%


In [ ]:
"""
Все пропуски в данном датасете я бы игонорировала. Поскольку они распределены случайно. 
Максимальная доля приходится на столбец distension, не превышает 20-ти %, однако заполнять их корректным не считаю. 
"""

In [33]:
# На примере столбца puls провела замену пропусков
df.puls.fillna(df.puls.median(), inplace=True)
df.puls.fillna(df.puls.median()).isna().sum()

0

In [34]:
# Удалим все строки с пропусками. Их число сократится с 300 до 233.
df.dropna().info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 233 entries, 0 to 298
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   surgery      233 non-null    float64
 1   age          233 non-null    int64  
 2   number       233 non-null    int64  
 3   puls         233 non-null    float64
 4   perestalsis  233 non-null    float64
 5   distension   233 non-null    float64
 6   outcome      233 non-null    float64
dtypes: float64(5), int64(2)
memory usage: 14.6 KB


In [25]:
# Рассчитать количество выбросов для всех выбранных столбцов. С этим заданием я не справилась...

"""
Я искренне не понимаю, что на выходе. Почему число значений всех столбцов меняется при каждой итерации.
Хочется уметь получить df, где в каждом из столбцов нет выбросов.
"""

for column in list(df):
    
        q1 = df[column].quantile(0.25)
        q3 = df[column].quantile(0.75)
        iqr = q3 - q1
        lower_bound = q1 - (1.5 * iqr) 
        upper_bound = q3 + (1.5 * iqr)
        print(f'{df[df[column].between(lower_bound, upper_bound, inclusive=True)].count()}')



surgery        299
age            299
number         299
puls           275
perestalsis    255
distension     243
outcome        299
dtype: int64
surgery        275
age            276
number         276
puls           254
perestalsis    238
distension     229
outcome        275
dtype: int64
surgery        263
age            264
number         264
puls           247
perestalsis    234
distension     218
outcome        263
dtype: int64
surgery        270
age            271
number         271
puls           271
perestalsis    240
distension     225
outcome        270
dtype: int64
surgery        216
age            217
number         217
puls           206
perestalsis    217
distension     201
outcome        216
dtype: int64
surgery        243
age            244
number         244
puls           229
perestalsis    234
distension     244
outcome        243
dtype: int64
surgery        299
age            299
number         299
puls           275
perestalsis    255
distension     243
outcome   